# 4DHumans Demo Notebook
This a demo notebook for our paper "4DHumans: Reconstructing and Tracking Humans with Transformers".

<p align="left"><img src="https://github.com/shubham-goel/4D-Humans/raw/main/assets/teaser.png" width="800"></p>

<p align="left"><img src="https://github.com/brjathu/PHALP/raw/master/assets/imgs/teaser.gif" width="800"></p>

Project webpage: https://shubham-goel.github.io/4dhumans/

Github repo: https://github.com/shubham-goel/4D-Humans

Instructions:

1. Enable the GPU Runtime (Runtime > Change Runtime Type > GPU)
2. Clone the repository and install depencencies
3. Run the demo
4. Visualize the video


# Clone the repository and install dependencies

In [ ]:
# Clone the main repo
%%capture
! git clone https://github.com/shubham-goel/4D-Humans.git 4D-Humans
%cd 4D-Humans

In [ ]:
!pip install torch
!pip install -e .[all]
!wget https://github.com/classner/up/raw/master/models/3D/basicModel_neutral_lbs_10_207_0_v1.0.0.pkl
!mkdir data/
!mv basicModel_neutral_lbs_10_207_0_v1.0.0.pkl data/

# Run the demo

In [ ]:
# run hmr2 on a folder of images
!python demo.py \
--img_folder example_data/images \
--out_folder demo_out \
--batch_size=48 --side_view --save_mesh

### Visualize the reconstructions

In [ ]:
from IPython.display import Image, display
import os
# https://colab.research.google.com/drive/1Ex4gE5v1bPR3evfhtG7sDHxQGsWwNwby?usp=sharing
output_images = ["demo_out/" + i for i in os.listdir("demo_out/") if ".png" in i]
for img in output_images:
  display(Image(img))



# Video Demo

### Install PHALP

In [ ]:
!pip install git+https://github.com/brjathu/PHALP.git

### Track and visualize

In [ ]:
!python track.py video.source="example_data/videos/gymnasts.mp4" video.start_frame=10 video.end_frame=60
# python track.py video.source="/scratch/ondemand27/evanpan/data/motion_tracking_data/nc_bigger_huge_biggest.mp4" 
# python track.py video.source="/scratch/ondemand27/evanpan/data/motion_tracking_data/truck_stand12.mp4" 
# python track.py video.source="/scratch/ondemand27/evanpan/data/motion_tracking_data/biggerspin123.mp4" 


In [1]:
# Display the reconstruction video
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                      <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

!ffmpeg -y -hide_banner -loglevel error -i outputs/PHALP_gymnasts.mp4 outputs/PHALP_gymnasts_ffmpeg.mp4
show_local_mp4_video('outputs/PHALP_gymnasts_ffmpeg.mp4', width=960, height=540)

^C


FileNotFoundError: [Errno 2] No such file or directory: 'outputs/PHALP_gymnasts_ffmpeg.mp4'